In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

data = pd.read_csv('../data/processed/dataset_reduced.csv')

# 对 Cation, Anion 进行One-hot编码
data_encoded = pd.get_dummies(data, columns=['Cation', 'Anion'])
# 划分训练集和测试集
data_encoded['target_class'] = pd.qcut(data_encoded['Cs'], q=10, labels=False)
X = data_encoded.drop(['Cs', 'target_class'], axis=1)
y = data_encoded['Cs']
stratify_column = data_encoded['target_class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=stratify_column)

In [2]:
# 将布尔值列转换为整数类型（True -> 1, False -> 0）
X_train = X_train.apply(lambda col: col.map(int) if col.dtype == 'bool' else col)
X_test = X_test.apply(lambda col: col.map(int) if col.dtype == 'bool' else col)

# 仅选择数值类型的列
X_train_num = X_train.select_dtypes(include=['number'])
X_test_num = X_test.select_dtypes(include=['number'])

# 将数据转换为NumPy数组，并确保是float32类型
X_train_tensor = torch.tensor(X_train_num.to_numpy(), dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32).view(-1, 1).to(device)
X_test_tensor = torch.tensor(X_test_num.to_numpy(), dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32).view(-1, 1).to(device)

# 创建DataLoader（批处理）
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# 定义ANN模型
class ANNModel(nn.Module):
    def __init__(self, input_dim, hidden_layers):
        super(ANNModel, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_layers:
            layers.append(nn.Linear(prev_dim, h_dim))
            layers.append(nn.ReLU())
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, 1))  # 输出层
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

In [3]:
# 定义MAPE损失函数
def mape_loss(y_pred, y_true):
    return torch.mean(torch.abs((y_true - y_pred) / y_true)) * 100  # 计算MAPE

# 定义目标函数
def objective(trial):
    # 超参数搜索：隐藏层数、每层神经元数量
    hidden_layers = []
    num_layers = trial.suggest_int("num_layers", 3, 6)  # 隐藏层数，范围为3到6层
    for i in range(num_layers):
        num_units = trial.suggest_int(f"num_units_{i}", 4, 128)  # 每层的神经元数量，范围为4到128
        hidden_layers.append(num_units)

    # 初始化模型并移动到GPU
    model = ANNModel(input_dim=X_train.shape[1], hidden_layers=hidden_layers).to(device)
    criterion = mape_loss
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # 训练模型
    model.train()
    for epoch in range(1000):
        running_loss = 0.0  # 用于记录每个epoch的平均损失
        for batch_X, batch_y in train_loader:
            # 将数据转移到GPU
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            optimizer.zero_grad()
            output = model(batch_X)
            loss = criterion(output, batch_y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        # 每50个epoch输出一次平均损失
        if (epoch + 1) % 50 == 0:
            avg_loss = running_loss / len(train_loader)
            print(f"Epoch [{epoch+1}/1000], Loss: {avg_loss:.4f}")

    # 在测试集上评估模型
    model.eval()
    with torch.no_grad():
        y_pred = model(X_test_tensor)
        loss = criterion(y_pred, y_test_tensor)

    return loss.item()

# 创建Optuna优化器
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

# 输出最优超参数
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print(f"  Params: {trial.params}")

[I 2025-03-12 19:20:56,661] A new study created in memory with name: no-name-c8fb1ef0-3a5b-494e-95ef-7b269727dee1


Epoch [50/1000], Loss: 49.1597
Epoch [100/1000], Loss: 41.3018
Epoch [150/1000], Loss: 35.1352
Epoch [200/1000], Loss: 31.1736
Epoch [250/1000], Loss: 22.9922
Epoch [300/1000], Loss: 21.2510
Epoch [350/1000], Loss: 21.1679
Epoch [400/1000], Loss: 20.0347
Epoch [450/1000], Loss: 15.3522
Epoch [500/1000], Loss: 15.3839
Epoch [550/1000], Loss: 12.7022
Epoch [600/1000], Loss: 12.5685
Epoch [650/1000], Loss: 10.9689
Epoch [700/1000], Loss: 11.7634
Epoch [750/1000], Loss: 10.4209
Epoch [800/1000], Loss: 10.0574
Epoch [850/1000], Loss: 8.7833
Epoch [900/1000], Loss: 10.7470
Epoch [950/1000], Loss: 8.7950


[I 2025-03-12 19:21:19,793] Trial 0 finished with value: 20.860591888427734 and parameters: {'num_layers': 4, 'num_units_0': 82, 'num_units_1': 65, 'num_units_2': 64, 'num_units_3': 83}. Best is trial 0 with value: 20.860591888427734.


Epoch [1000/1000], Loss: 8.6052
Epoch [50/1000], Loss: 49.1181
Epoch [100/1000], Loss: 39.4044
Epoch [150/1000], Loss: 34.2439
Epoch [200/1000], Loss: 29.3889
Epoch [250/1000], Loss: 25.7302
Epoch [300/1000], Loss: 23.0474
Epoch [350/1000], Loss: 21.5791
Epoch [400/1000], Loss: 19.8034
Epoch [450/1000], Loss: 17.5466
Epoch [500/1000], Loss: 16.6926
Epoch [550/1000], Loss: 15.1142
Epoch [600/1000], Loss: 16.4807
Epoch [650/1000], Loss: 14.1499
Epoch [700/1000], Loss: 14.7404
Epoch [750/1000], Loss: 13.0082
Epoch [800/1000], Loss: 12.6437
Epoch [850/1000], Loss: 11.9381
Epoch [900/1000], Loss: 12.4344
Epoch [950/1000], Loss: 11.2997


[I 2025-03-12 19:21:38,440] Trial 1 finished with value: 21.84808349609375 and parameters: {'num_layers': 3, 'num_units_0': 104, 'num_units_1': 25, 'num_units_2': 110}. Best is trial 0 with value: 20.860591888427734.


...

Epoch [1000/1000], Loss: 5.5136
Epoch [50/1000], Loss: 42.5576
Epoch [100/1000], Loss: 31.6337
Epoch [150/1000], Loss: 23.6387
Epoch [200/1000], Loss: 19.8699
Epoch [250/1000], Loss: 15.2065
Epoch [300/1000], Loss: 14.2743
Epoch [350/1000], Loss: 10.5629
Epoch [400/1000], Loss: 10.0637
Epoch [450/1000], Loss: 10.2715
Epoch [500/1000], Loss: 7.6068
Epoch [550/1000], Loss: 7.2428
Epoch [600/1000], Loss: 8.8904
Epoch [650/1000], Loss: 7.4778
Epoch [700/1000], Loss: 7.3451
Epoch [750/1000], Loss: 5.9056
Epoch [800/1000], Loss: 5.3985
Epoch [850/1000], Loss: 6.1963
Epoch [900/1000], Loss: 5.3034
Epoch [950/1000], Loss: 5.3437


[I 2025-03-12 20:00:53,818] Trial 98 finished with value: 18.614818572998047 and parameters: {'num_layers': 6, 'num_units_0': 42, 'num_units_1': 93, 'num_units_2': 116, 'num_units_3': 109, 'num_units_4': 84, 'num_units_5': 115}. Best is trial 69 with value: 12.174188613891602.


Epoch [1000/1000], Loss: 5.2554
Epoch [50/1000], Loss: 43.7806
Epoch [100/1000], Loss: 35.5549
Epoch [150/1000], Loss: 24.4774
Epoch [200/1000], Loss: 21.6697
Epoch [250/1000], Loss: 16.9402
Epoch [300/1000], Loss: 12.3718
Epoch [350/1000], Loss: 11.9422
Epoch [400/1000], Loss: 12.9622
Epoch [450/1000], Loss: 11.3503
Epoch [500/1000], Loss: 7.7084
Epoch [550/1000], Loss: 8.7721
Epoch [600/1000], Loss: 7.5512
Epoch [650/1000], Loss: 7.8368
Epoch [700/1000], Loss: 6.4369
Epoch [750/1000], Loss: 5.7667
Epoch [800/1000], Loss: 6.1725
Epoch [850/1000], Loss: 5.2001
Epoch [900/1000], Loss: 5.6266
Epoch [950/1000], Loss: 6.1707


[I 2025-03-12 20:01:18,905] Trial 99 finished with value: 20.364547729492188 and parameters: {'num_layers': 6, 'num_units_0': 58, 'num_units_1': 120, 'num_units_2': 68, 'num_units_3': 126, 'num_units_4': 115, 'num_units_5': 103}. Best is trial 69 with value: 12.174188613891602.


Epoch [1000/1000], Loss: 6.0821
Best trial:
  Value: 12.174188613891602
  Params: {'num_layers': 6, 'num_units_0': 126, 'num_units_1': 98, 'num_units_2': 101, 'num_units_3': 125, 'num_units_4': 109, 'num_units_5': 89}
